In [ ]:
import importlib
import local_utils; importlib.reload(local_utils)
from local_utils import *

train_df = pd.read_csv(DATA_DIR + '/train_v2.csv')
test_df = pd.read_csv(DATA_DIR + '/sample_submission_v2.csv')

label_map, inv_label_map, Y = process_labels(train_df)
print("Shape of Y {}".format(Y.shape))

train_inx, valid_inx = stratified_sampling(Y)
print("Train {}; Valid {};".format(len(train_inx), len(valid_inx)))

w_size = 128
h_size = 128
input_shape = (w_size, h_size, 3)
print(input_shape)

In [ ]:
start = time()
X = load_images(train_df['image_name'].values, DATA_DIR + '/train-jpg', True, w_size, h_size)
print(X.shape)
print(time() - start)

In [5]:
model = Sequential()
model.add(BatchNormalization(input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

In [6]:
batch_size = 64
model.compile(optimizer=optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 63, 63, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 61, 61, 64)        36928     
__________

In [9]:
model.fit(X[train_inx], Y[train_inx],
          batch_size=batch_size,
          epochs=3,
          verbose=1,
          validation_data=(X[valid_inx], Y[valid_inx]))

Train on 36431 samples, validate on 4048 samples
Epoch 1/1
36431/36431 [==============================] - 157s - loss: 0.1494 - acc: 0.9409 - val_loss: 0.1462 - val_acc: 0.9433


In [10]:
model.optimizer.lr = 1e-4
model.fit(X[train_inx], Y[train_inx],
          batch_size=batch_size,
          epochs=3,
          verbose=1,
          validation_data=(X[valid_inx], Y[valid_inx]))

Train on 36431 samples, validate on 4048 samples
Epoch 1/3
36431/36431 [==============================] - 157s - loss: 0.1437 - acc: 0.9429 - val_loss: 0.1380 - val_acc: 0.9449
Epoch 2/3
36431/36431 [==============================] - 158s - loss: 0.1387 - acc: 0.9451 - val_loss: 0.1288 - val_acc: 0.9504
Epoch 3/3
36431/36431 [==============================] - 158s - loss: 0.1338 - acc: 0.9476 - val_loss: 0.1244 - val_acc: 0.9507


In [12]:
model.optimizer.lr = 1e-5
model.fit(X[train_inx], Y[train_inx],
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X[valid_inx], Y[valid_inx]))

Train on 36431 samples, validate on 4048 samples
Epoch 1/2
36431/36431 [==============================] - 158s - loss: 0.1185 - acc: 0.9540 - val_loss: 0.1355 - val_acc: 0.9496
Epoch 2/2
36431/36431 [==============================] - 158s - loss: 0.1160 - acc: 0.9552 - val_loss: 0.1123 - val_acc: 0.9567


In [13]:
model.save(DATA_DIR + '/models/simple_model.h5', include_optimizer=False)

In [24]:
save_array(DATA_DIR + '/models/simple_model_thres.dat', thres)